In [1]:
# in the class imbalance ipynb we figured out that we will not over/undersample the data

# loading the preprcessed dataset
import pandas as pd

df = pd.read_csv("/Users/mv96/Desktop/dataset_tkb/test.csv")

print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'page_no', 'top_left', 'bot_right',
       'grobid_text', 'pdf_alto_text', 'fonts', 'label', 'Normal',
       'Superscipt', 'Subscript', 'italics', 'bold', 'is_Proportional',
       'is_Serif', 'font_color_red', 'font_color_green', 'font_color_blue',
       'is_bold_manual', 'is_italic_manual', 'is_serif_manual',
       'is_math_manual', 'new_font_size', 'pdf_path'],
      dtype='object')


In [2]:
cols_list = list(filter(lambda x: not x.startswith("Unnamed"), list(df.columns)))
font_vectors = df[cols_list].iloc[:, 7:-1]
labels = df["label"]

In [3]:
font_vectors.describe()

,Normal,Superscipt,Subscript,italics,bold,is_Proportional,is_Serif,font_color_red,font_color_green,font_color_blue,is_bold_manual,is_italic_manual,is_serif_manual,is_math_manual,new_font_size
count,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000,388519.000000
mean,0.905200,0.027543,0.049060,0.017119,0.002733,0.970726,0.000311,0.000459,0.000361,0.000445,0.178836,0.212708,0.269140,0.069849,0.259846
std,0.129627,0.053105,0.069949,0.087168,0.028547,0.155933,0.015420,0.009692,0.007635,0.009782,0.331913,0.235538,0.383401,0.102905,0.021880
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.862745,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013580,0.000000,0.000000,0.247830
50%,0.947368,0.000000,0.017857,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.139425,0.009091,0.030556,0.264786
75%,1.000000,0.034799,0.077193,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.103874,0.315789,0.633333,0.100000,0.272725
max,1.000000,1.000000,0.854342,1.000000,1.000000,1.000000,1.000000,0.997099,0.780882,1.000000,1.000000,1.000000,1.000000,1.000000,0.619675


In [4]:
combined = pd.concat([font_vectors, labels], axis=1)
combined.shape

(388809, 16)

In [5]:
# removing nan values
combined = combined.dropna()
combined.shape

(388519, 16)

In [6]:
X = combined.iloc[:, :-1]
y = combined["label"]

# divide the dataset into test and train portions
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42, stratify=y
)

print(X.shape, y.shape)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(388519, 15) (388519,)
(349667, 15) (38852, 15) (349667,) (38852,)


In [28]:
# we will now use recursive feature selection to see the impact over three classifiers

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

import numpy as np
from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import SequentialFeatureSelector
import copy

clfs = [KNeighborsClassifier(), MultinomialNB(), LogisticRegression(random_state=0)]

for clf in tqdm(clfs):
    print(clf)
    print("=" * 20)
    print("before selecting")
    clff = copy.deepcopy(clf)
    ##default settings

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    print(acc, f1)

    print("=" * 20)
    print("selecting n best features")
    sfs = SequentialFeatureSelector(clf, n_features_to_select=7, n_jobs=4, cv=3)
    sfs.fit(X_train, y_train)
    X_train_transformed = sfs.transform(X_train)
    X_test_transformed = sfs.transform(X_test)
    print(X_transformed.shape)

    print("=" * 20)
    print("running clf on n best features or reduced data")
    clff.fit(X_train_transformed, y_train)
    y_pred = clff.predict(X_test_transformed)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    print(acc, f1)
    print(sfs.get_support())

  0%|                                                     | 0/3 [00:00<?, ?it/s]

KNeighborsClassifier()
before selecting


/Users/mv96/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.5745907546587048 0.5678586420596728
selecting n best features
(349667, 3)
running clf on n best features or reduced data


 33%|██████████████▎                            | 1/3 [18:01<36:02, 1081.39s/it]

...

/Users/mv96/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mv96/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

/Users/mv96/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mv96/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

(349667, 3)
running clf on n best features or reduced data


/Users/mv96/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
100%|████████████████████████████████████████████| 3/3 [25:16<00:00, 505.37s/it]

0.5832132193966848 0.5459551843781923
[ True  True  True False False False False False False False  True  True
 False  True  True]


In [29]:
#so from selecting 7 features we dont really lose much of the information
important_features= [for ele1,ele2 in zip()]

[ True  True  True False False False False False False False  True  True
 False  True  True]
